In [126]:
import pickle,sys
from nltk import PorterStemmer
from math import sqrt as sqrt
from collections import defaultdict
import codecs
import re
import math
import functools
import pickle

In [127]:
postingdict = defaultdict(dict)

In [128]:
with open("postinglist.pkl","rb") as f:
    postingdict = pickle.load(f)

In [129]:
total = len(postingdict)

In [130]:
doc_n = 100

In [131]:
def idfcal(token):
	idf=0.0
	if(token in postingdict):
		idf=math.log(doc_n/(len(postingdict[token])+1),10)
	return(idf)

In [132]:
def twcal(token,docid):
	tw=0.0
	if(docid in postingdict[token]):
		idf=idfcal(token)
		tf=1+math.log(postingdict[token][docid])
		tw=tf*idf
	return(tw)

In [133]:
def create_length_doc():
    length_doc = {}
    for docid in range(doc_n):
        l=0
        for token in postingdict.keys():
            tw=twcal(token,docid)
            l=l+(tw**2)
        length_doc[docid]=math.sqrt(l)
    return length_doc
# length_doc = create_length_doc()

In [134]:
# with open( "engdoclen" + '.pkl', 'wb') as f:
#         pickle.dump(length_doc, f, pickle.HIGHEST_PROTOCOL)

In [135]:
length_doc=dict()
with open("engdoclen.pkl","rb") as f:
    length_doc = pickle.load(f)

In [136]:
def cosine(query,docid):
	sc=0
	for token in query:
		if token in postingdict.keys():
			tw=twcal(token,docid)
			sc=sc + (tw)
	sc=sc/(length_doc[docid])
	return(sc)

In [137]:
def isStopword(word):
    f=codecs.open('en.stopword.txt','r',encoding="utf-8")
    stopwords = f.read().split("\n")
    #print(stopwords)
    return True if (word.lower() in stopwords) else False

In [138]:
def proc_query(qry):
    query=qry.split(" ")
    docset_of_terms=[]
    stemmer=PorterStemmer()
    
    for tokenword in query:
        tokenword = stemmer.stem(tokenword.lower())
        docset_of_terms.append( set(postingdict[tokenword].keys()) )
    relevant_docs = functools.reduce(set.union,[l for l in docset_of_terms])
    
    if not relevant_docs:
        print("Error : No relevant documents found for the query\n")
        return
    else:
        score=sorted([(docid,cosine(query,docid)) for docid in relevant_docs])
        score=sorted(score,key=lambda x:x[1],reverse=True)
        return score

In [139]:
ans=proc_query('cricket world cup')
for doc in ans[:9]:
    print(str(doc[0]) + " --- " + str(doc[1]) )

60 --- 0.10302524629694818
61 --- 0.09218378970193572
70 --- 0.08914152559343162
31 --- 0.07758772411361396
36 --- 0.057543511755790686
90 --- 0.05388312560921563
43 --- 0.0528002730387565
28 --- 0.04821361449114888
52 --- 0.04790462914465872


In [140]:
hub = []
auth = []
for doc in ans[:10]:
    hub.append(doc[0])
    auth.append(doc[0])

In [141]:
outlinks = pickle.load(open('adjacencymatrix.pkl', 'rb'))

In [142]:
for i in hub:
    for j in range(100):
            if (outlinks[i][j] == 1 or outlinks[j][i] == 1) and j not in auth:
                auth.append(j)

In [143]:
print("saving query induced graph hub and auth pages list query = cricket world cup\n")
with open( "query_hub_auth" + '.pkl', 'wb') as f:
        pickle.dump(auth, f, pickle.HIGHEST_PROTOCOL)

saving query induced graph hub and auth pages list query = cricket world cup

